In [1]:
%pip install langchain-core langgraph>0.2.27

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -qU "langchain[mistralai]"

Note: you may need to restart the kernel to use updated packages.


In [3]:
import getpass
import os

if not os.environ.get("MISTRAL_API_KEY"):
  os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("mistral-large-latest", model_provider="mistralai")

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hello Bob! Nice to meet you. How are you today? Let's chat about anything you'd like. 😊", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 9, 'total_tokens': 35, 'completion_tokens': 26}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-5a94e86c-b405-4baa-9db3-a72a5d4f0833-0', usage_metadata={'input_tokens': 9, 'output_tokens': 26, 'total_tokens': 35})

In [5]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm an artificial intelligence and I don't have real-time personal information about you. I can only see and respond to the text you've typed. If you'd like me to use a specific name when addressing you, please tell me!", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 9, 'total_tokens': 62, 'completion_tokens': 53}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-6856283a-1654-4ea5-a7c4-b446056acf60-0', usage_metadata={'input_tokens': 9, 'output_tokens': 53, 'total_tokens': 62})

In [6]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='You told me your name is Bob! Is that correct?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 28, 'total_tokens': 40, 'completion_tokens': 12}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-b1bde22e-275e-4914-a38e-7e8998aab954-0', usage_metadata={'input_tokens': 28, 'output_tokens': 12, 'total_tokens': 40})

In [7]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [13]:
config = {"configurable": {"thread_id": "abc123"}}

In [14]:
query = "Hi! I'm Bob. Whenever call me call me Bob. No additional information is needed."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Understood, Bob! I'll make sure to call you Bob. Let's talk about whatever you'd like, Bob. How's your day going so far?


In [15]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Bob, as you instructed, I will call you Bob. Is there anything you need help with today, Bob?


In [18]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

TypeError: No synchronous function provided to "model".
Either initialize with a synchronous function or invoke via the async API (ainvoke, astream, etc.)